In [1]:
from protosc.simulation import create_categorical_data, compare_results
from protosc.model import WrapperModel
from protosc.model.genetic import GeneticModel
from protosc.model.utils import train_kfold_validate
from protosc.model import CombinedFoldModel
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
X, y, ground_truth = create_categorical_data(min_dev=0.25, max_dev=0.5, n_categories=3, seed=1234)
print(f'ground truth #features: {len(ground_truth["selected_features"])}')
# X, y, ground_truth = create_correlated_data()

ground truth #features: 25


In [3]:
train_kfold_validate(X, y, ground_truth["selected_features"])

0.5964334128275692

### Run all models

In [4]:
model = CombinedFoldModel()
results = model.execute(X, y, progress_bar=True, n_jobs=-1)

100%|██████████| 8/8 [00:16<00:00,  2.04s/it]


In [5]:
for name, res in results.items():
    print(name)
    print(compare_results(res, ground_truth))

filter
{'%corr_feat': 0.35135135135135137, '%feat_found': 0.52, '%bias_found': 0.5455555555555556}
fast_wrapper
{'%corr_feat': 0.35294117647058826, '%feat_found': 0.48, '%bias_found': 0.49}
slow_wrapper
{'%corr_feat': 0.37037037037037035, '%feat_found': 0.4, '%bias_found': 0.4522222222222222}
random
{'%corr_feat': 0, '%feat_found': 0.0, '%bias_found': 0.0}
pseudo_random
{'%corr_feat': 0, '%feat_found': 0.0, '%bias_found': 0.0}


In [6]:
gen_features = GeneticModel(n_gen_data=5, num_penalty=0.0001).execute(X, y, n_jobs=-1)

Acc: 0.51 (max: 0.58, #f: 50): 100%|██████████| 30/30 [02:47<00:00,  5.59s/it]


In [7]:
compare_results(gen_features, ground_truth)

{'%corr_feat': 0.7142857142857143,
 '%feat_found': 0.4,
 '%bias_found': 0.4422222222222222}